<a href="https://colab.research.google.com/github/MazedaZ/Pre-trained-ANN-to-SNN-conversion/blob/main/Converting_Pre_trained_ANNs_to_SNNs_for_Efficient_Neuromorphic_Computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Define a basic ANN model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Define a simple spiking model (SNN)
class SpikingModel(nn.Module):
    def __init__(self):
        super(SpikingModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        spikes = self.fc2(x)  # We use the output spikes directly
        return spikes

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize ANN and SNN models
ann_model = ANNModel()
snn_model = SpikingModel()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
ann_optimizer = torch.optim.SGD(ann_model.parameters(), lr=0.01)
snn_optimizer = torch.optim.SGD(snn_model.parameters(), lr=0.01)

# Training loop for ANN and SNN
def train(model, optimizer, criterion, train_loader, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

# Train ANN and SNN
train(ann_model, ann_optimizer, criterion, train_loader)
train(snn_model, snn_optimizer, criterion, train_loader)

# Evaluate the models (accuracy)
def test(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Load the test dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Test ANN and SNN accuracy
ann_accuracy = test(ann_model, test_loader)
snn_accuracy = test(snn_model, test_loader)

print(f"ANN Accuracy: {ann_accuracy:.2f}%")
print(f"SNN Accuracy: {snn_accuracy:.2f}%")


100%|██████████| 9912422/9912422 [00:00<00:00, 19815395.28it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 67863133.79it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28757859.66it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12649753.50it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1, Loss: 0.741627476902913
Epoch 2, Loss: 0.3644565206283191
Epoch 3, Loss: 0.32032417711704525
Epoch 4, Loss: 0.29409850422126144
Epoch 5, Loss: 0.2726819357558735
Epoch 1, Loss: 0.6878125261364461
Epoch 2, Loss: 0.3688239461180371
Epoch 3, Loss: 0.33279449928766375
Epoch 4, Loss: 0.31731965489733194
Epoch 5, Loss: 0.3073501154335577
ANN Accuracy: 92.75%
SNN Accuracy: 91.63%
